In [ ]:
#Problem Statement..

Data scientist is the sexiest job in the world. How many times have you heard that? Analytics India Annual Salary Study which aims to understand a wide range of trends data science says that the median analytics salary in India for the year 2017 is INR 12.7 Lakhs across all experience level and skill sets. So given the job description and other key information can you predict the range of salary of the job posting? What kind of factors influence the salary of a data scientist? The study also says that in the world of analytics, Mumbai is the highest paymaster at almost 13.3 Lakhs per annum, followed by Bengaluru at 12.5 Lakhs. The industry of the data scientist can also influence the salary. Telecom industry pays the highest median salaries to its analytics professionals at 18.6 Lakhs. What are you waiting for, solve the problem by predicting how much a data scientist or analytics professional will be paid by analysing the data given. Bonus Tip: You can analyse the data and get key insights for your career as well. The best data scientists and machine learning engineers will be given awesome prizes at the end of hackathon. Share this hackathon with a colleague who may be interested in mining the dataset for insights and make great predictions. Data The dataset is based on salary and job postings in India across the internet. The train and the test data consists of attributes mentioned below. The rows of train dataset has rich amount of information regarding the job posting such as name of the designation and key skills required for the job. The training data and test data comprise of 19802 samples and of 6601 samples each. This is a dataset which has been collected over some time to gather relevant analytics jobs posting over the years. Features Name of the company (Encoded) Years of experience Job description Job designation Job Type Key skills Location Salary in Rupees Lakhs(To be predicted) Problem Statement Based on the given attributes and salary information, build a robust machine learning model that predicts the salary range of the salary post.

# Acheived Rank 8 in this Machine Learning Hackathon

In [1]:
#IMPORTING The Necessary Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [41]:
train_data=pd.read_csv('Final_Train_Dataset.csv',usecols=['experience', 'job_description', 'job_desig', 'job_type',
       'key_skills', 'location', 'salary', 'company_name_encoded'])
test_data=pd.read_csv('Final_Test_Dataset.csv')

In [42]:
train_data.head(2)

,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458


In [43]:
test_data.head(2)

,experience,job_description,job_desig,job_type,key_skills,location,company_name_encoded
0,7-12 yrs,Professional experience in Java/J2EE based ser...,IT Technology Senior Consultant/java/ J2ee/ Se...,NaN,"Java, J2Ee, Tomcat, JBoss, Weblogic, Oracle, E...",Bengaluru,2066
1,0-5 yrs,We are looking for 20+ Fresher/Experienced Can...,Medical Billing Process | International KPO | ...,NaN,"Medical Billing, Insurance Processing",Ahmedabad(Sola),2629


In [44]:
train_data['job_type'].fillna('missingjobtype', inplace=True)
train_data['job_type'].replace('Analytics', 'analytics', inplace=True)
train_data['job_type'].replace('Analytic', 'analytics', inplace=True)
train_data['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
train_data['job_type'].replace('analytic', 'analytics', inplace=True)

test_data['job_type'].fillna('missingjobtype', inplace=True)
test_data['job_type'].replace('Analytics', 'analytics', inplace=True)
test_data['job_type'].replace('Analytic', 'analytics', inplace=True)
test_data['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
test_data['job_type'].replace('analytic', 'analytics', inplace=True)

In [45]:
train_data.dropna(subset=['key_skills'],inplace=True)
test_data.dropna(subset=['key_skills'],inplace=True)

In [46]:
#For Descriptions which have Nan Values , we would replace them with the "Missing Description"
train_data['job_description'].fillna("Missing Description",inplace=True)
#For Descriptions which have Nan Values , we would replace them with the "Missing Description"
test_data['job_description'].fillna("Missing Description",inplace=True)

In [47]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19801 entries, 0 to 19801
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   experience            19801 non-null  object
 1   job_description       19801 non-null  object
 2   job_desig             19801 non-null  object
 3   job_type              19801 non-null  object
 4   key_skills            19801 non-null  object
 5   location              19801 non-null  object
 6   salary                19801 non-null  object
 7   company_name_encoded  19801 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 1.4+ MB


In [48]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6601 entries, 0 to 6600
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   experience            6601 non-null   object
 1   job_description       6601 non-null   object
 2   job_desig             6601 non-null   object
 3   job_type              6601 non-null   object
 4   key_skills            6601 non-null   object
 5   location              6601 non-null   object
 6   company_name_encoded  6601 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 412.6+ KB


In [49]:
train_data['Minimum_experience']=train_data['experience'].apply(lambda x:x.split('-')[0])
train_data['Maximum_experience']=train_data['experience'].apply(lambda x:x.split('-')[1])
train_data['Maximum_experience']=train_data['Maximum_experience'].apply(lambda x:x.split(' ')[0])

In [50]:
test_data['Minimum_experience']=test_data['experience'].apply(lambda x:x.split('-')[0])
test_data['Maximum_experience']=test_data['experience'].apply(lambda x:x.split('-')[1])
test_data['Maximum_experience']=test_data['Maximum_experience'].apply(lambda x:x.split(' ')[0])

In [51]:
#cleaning unecessary tokens in skills for better understanding for algorithms...
def clean_skills(skill):
    skills=str(skill).lower()
    skills=re.sub(r'\...',' ',skills)
    skills=re.sub(r',',' ',skills)
    skills=re.sub('\s+',' ',skills)
    return skills

train_data['Key_Skills']=train_data['key_skills'].apply(clean_skills)
test_data['Key_Skills']=test_data['key_skills'].apply(clean_skills)

In [52]:
train_data['Key_Skills'][0],train_data['Key_Skills'][1]

('team skills communication skills analytical skills problem solving ',
 'ppc logistics inventory management supply chain management procurement ')

In [53]:
#Since there are a lot of unnecessary tokens in the text, we  have to clean those for further better understanding , so that 
#algorithms learn our data better...
def clean_description(description):
    desc=str(description).lower()
    desc=re.sub(r',',' ',desc)
    desc=re.sub(r'/...',' ',desc)
    desc=re.sub(r'[^a-z]',' ',desc)
    desc=re.sub('\s+',' ',desc)
    return desc


train_data['JOB_DESCRIPTION']=train_data['job_description'].apply(clean_description)
test_data['JOB_DESCRIPTION']=test_data['job_description'].apply(clean_description)

In [54]:
def clean_desig(desig):
    designation=str(desig).lower()
    designation=re.sub('[^a-z]',' ',designation)
    designation=re.sub('\s+',' ',designation)
    return designation

train_data['Job_Desig']=train_data['job_desig'].apply(clean_desig)
test_data['Job_Desig']=test_data['job_desig'].apply(clean_desig)

In [55]:
def clean_location(loc):
    location = loc.lower()
    location = re.sub(r'[^a-z]', ' ', location)
    location = re.sub(r'\s+', ' ', location)
    return location

train_data['Location'] = train_data['location'].apply(clean_location)
test_data['Location'] = test_data['location'].apply(clean_location)

In [56]:
train_data.drop(['job_description','job_desig','location','key_skills'],1,inplace=True)
test_data.drop(['job_description','job_desig','location','key_skills'],1,inplace=True)

In [57]:
train_data.drop(['experience'],1,inplace=True)
test_data.drop(['experience'],1,inplace=True)

In [58]:
train_data.head(2)

,job_type,salary,company_name_encoded,Minimum_experience,Maximum_experience,Key_Skills,JOB_DESCRIPTION,Job_Desig,Location
0,missingjobtype,6to10,3687,5,7,team skills communication skills analytical sk...,exp minimum years good understanding of ioc ru...,senior exploit and vulnerability researcher,delhi ncr vikas puri
1,missingjobtype,10to15,458,10,17,ppc logistics inventory management supply chai...,he should have handled a team of atleast direc...,head scm,sonepat


In [59]:
#Do Exploratory Data Analysis

In [60]:
train_data['merged_text']=(train_data['Job_Desig'] + ' ' + train_data['JOB_DESCRIPTION'] + ' ' + train_data['Key_Skills']
                      + ' ' + train_data['job_type'])

test_data['merged_text']=(test_data['Job_Desig'] + ' ' + test_data['JOB_DESCRIPTION'] + ' ' + test_data['Key_Skills']
                      + ' ' + test_data['job_type'])

In [61]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['salary'] = le.fit_transform(train_data['salary'])

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_data[['merged_text', 'Location','Minimum_experience','Maximum_experience','company_name_encoded']], 
    train_data['salary'], test_size=0.20, 
    stratify=train_data['salary'], random_state=75)

In [63]:
X_train_merged = X_train['merged_text']
X_train_location = X_train['Location']

X_test_merged = X_test['merged_text']
X_test_location = X_test['Location']

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [65]:
tf1 = TfidfVectorizer(min_df=3, token_pattern=r'\w{3,}', ngram_range=(1,3), max_df=0.9,stop_words='english')
tf2 = TfidfVectorizer(min_df=2, token_pattern=r'\w{3,}',stop_words='english')

X_train_merged = tf1.fit_transform(X_train_merged)
X_train_location = tf2.fit_transform(X_train_location)

X_test_merged = tf1.transform(X_test_merged)
X_test_location = tf2.transform(X_test_location)

In [66]:
from scipy import sparse
from sklearn.preprocessing import StandardScaler

sc1 = StandardScaler()
X_train_MinExp = sc1.fit_transform(np.array(X_train['Minimum_experience']).reshape(-1,1))
X_test_MinExp = sc1.transform(np.array(X_test['Minimum_experience']).reshape(-1,1))
X_train_MinExp = sparse.csr_matrix(X_train_MinExp)
X_test_MinExp = sparse.csr_matrix(X_test_MinExp)

sc2 = StandardScaler()
X_train_MaxExp = sc2.fit_transform(np.array(X_train['Maximum_experience']).reshape(-1,1))
X_test_MaxExp = sc2.transform(np.array(X_test['Maximum_experience']).reshape(-1,1))
X_train_MaxExp = sparse.csr_matrix(X_train_MaxExp)
X_test_MaxExp = sparse.csr_matrix(X_test_MaxExp)

sc3 = StandardScaler()
X_train_comp = sc3.fit_transform(np.array(X_train['company_name_encoded']).reshape(-1,1))
X_test_comp = sc3.transform(np.array(X_test['company_name_encoded']).reshape(-1,1))
# X_train_comp=np.array(X_train['company_name_encoded']).reshape(-1,1)
# X_test_comp=np.array(X_test['company_name_encoded']).reshape(-1,1)

X_train_comp = sparse.csr_matrix(X_train_comp)
X_test_comp = sparse.csr_matrix(X_test_comp)

In [67]:
from scipy.sparse import hstack, csr_matrix
merged_train = hstack((X_train_merged, X_train_location, X_train_MinExp, X_train_MaxExp,X_train_comp))
merged_test  = hstack((X_test_merged, X_test_location, X_test_MinExp, X_test_MaxExp,X_test_comp))

In [68]:
merged_train.shape, merged_test.shape

((15840, 46913), (3961, 46913))

In [69]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [70]:
import lightgbm as lgb
train = lgb.Dataset(merged_train, label=y_train)
test = lgb.Dataset(merged_test, label=y_test)

In [71]:
# train_data

In [72]:
param = {'objective': 'multiclass',
         'num_iterations': 80,
         'learning_rate': 0.03,  
         'num_leaves': 15,
         'max_depth': 15, 
         'min_data_in_leaf': 25, 
         'max_bin': 20, 
         'min_data_in_bin': 3,   
         'num_class': 6,
         'metric': 'multi_logloss'
         }

In [73]:
lgbm = lgb.train(params=param,
                 train_set=train,
                 num_boost_round=100,
                 valid_sets=[test])

y_pred_class = lgbm.predict(merged_test)

C:\Users\welcome\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.099541 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46391
[LightGBM] [Info] Number of data points in the train set: 15840, number of used features: 3064
[LightGBM] [Info] Start training from score -1.808567
[LightGBM] [Info] Start training from score -1.481605
[LightGBM] [Info] Start training from score -1.568616
[LightGBM] [Info] Start training from score -2.531427
[LightGBM] [Info] Start training from score -1.947616
[LightGBM] [Info] Start training from score -1.724030
[1]	valid_0's multi_logloss: 1.71695
[2]	valid_0's multi_logloss: 1.6906
[3]	valid_0's multi_logloss: 1.66625
[4]	valid_0's multi_logloss: 1.64429
[5]	valid_0's multi_logloss: 1.62362
[6]	valid_0's multi_logloss: 1.60471
[7]	valid_0's multi_logloss: 1.58709
[8]	valid_0's multi_logloss: 1.57071
[9]	valid_0's multi_logloss: 1.55471
[10]	valid_0's multi_logloss: 1.53993
[11]	valid_

C:\Users\welcome\anaconda3\lib\site-packages\lightgbm\basic.py:597: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [74]:
predictions = []
for x in y_pred_class:
    predictions.append(np.argmax(x))

print('accuracy:', accuracy_score(y_test, predictions))

accuracy: 0.4822014642766978


In [ ]:
X_train_merged = train_data['merged_text']
X_train_location = train_data['Location']

X_test_merged =test_data['merged_text']
X_test_location =test_data['Location']

y_train = train_data['salary']

In [ ]:
tf1 = TfidfVectorizer(min_df=3, token_pattern=r'\w{3,}', ngram_range=(1,3))
tf2 = TfidfVectorizer(min_df=2, token_pattern=r'\w{3,}',ngram_range=(1,2))

X_train_merged = tf1.fit_transform(X_train_merged)
X_train_location = tf2.fit_transform(X_train_location)

X_test_merged = tf1.transform(X_test_merged)
X_test_location = tf2.transform(X_test_location)

In [ ]:
from scipy import sparse
from sklearn.preprocessing import StandardScaler

sc1 = StandardScaler()
X_train_MinExp = sc1.fit_transform(np.array(train_data['Minimum_experience']).reshape(-1,1))
X_test_MinExp = sc1.transform(np.array(test_data['Minimum_experience']).reshape(-1,1))
X_train_MinExp = sparse.csr_matrix(X_train_MinExp)
X_test_MinExp = sparse.csr_matrix(X_test_MinExp)

sc2 = StandardScaler()
X_train_MaxExp = sc2.fit_transform(np.array(train_data['Maximum_experience']).reshape(-1,1))
X_test_MaxExp = sc2.transform(np.array(test_data['Maximum_experience']).reshape(-1,1))
X_train_MaxExp = sparse.csr_matrix(X_train_MaxExp)
X_test_MaxExp = sparse.csr_matrix(X_test_MaxExp)

sc3 = StandardScaler()
X_train_comp = sc3.fit_transform(np.array(train_data['company_name_encoded']).reshape(-1,1))
X_test_comp = sc3.transform(np.array(test_data['company_name_encoded']).reshape(-1,1))
X_train_comp = sparse.csr_matrix(X_train_comp)
X_test_comp = sparse.csr_matrix(X_test_comp)

In [ ]:
merged_train = hstack((X_train_merged, X_train_location, X_train_MinExp, X_train_MaxExp,X_train_comp))
merged_test  = hstack((X_test_merged, X_test_location, X_test_MinExp, X_test_MaxExp,X_test_comp))

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(merged_train, label=y_train)

param = {'objective': 'multiclass',
         'num_iterations': 80,
         'learning_rate': 0.03,  
         'num_leaves': 15,
         'max_depth': 15, 
         'min_data_in_leaf': 25, 
         'max_bin': 20, 
         'min_data_in_bin': 3,   
         'num_class': 6,
         'metric': 'multi_logloss'
         }

lgbm = lgb.train(params=param, 
                 train_set=train_data)

predictions = lgbm.predict(merged_test)

y_pred_class = []
for x in predictions:
    y_pred_class.append(np.argmax(x))

y_pred_class = le.inverse_transform(y_pred_class)

In [ ]:
df_sub = pd.DataFrame(data=y_pred_class, columns=['salary'])

In [ ]:
writer = pd.ExcelWriter('output3.xlsx', engine='xlsxwriter')
df_sub.to_excel(writer,sheet_name='Sheet1', index=False)
writer.save()